In [ ]:
import os
import data
import model
import utils
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from IPython.display import clear_output
from pprint import pprint

In [ ]:
data.load()
hyper_params = {
    'batch_size': 28,
    'num_features': 20 * 80,
    'num_hiddens_fc': [3200, 3200, 1600],
    'num_classes': 2,
    'learning_rate': 1e-2
}

In [ ]:
tf.reset_default_graph()
graph, model_vars = model.create_graph(hyper_params)

In [ ]:
train_params = {
    'batch_size' : 2,
    'skip_step' : 4,
    'n_epochs' : 10000000
}

with tf.Session(graph=graph) as sess:
    # Mot vai thu tuc
    tf.global_variables_initializer().run()
    writer = tf.summary.FileWriter('./graphs/', sess.graph)
    saver = tf.train.Saver()
    ckpt = tf.train.get_checkpoint_state('./checkpoints/')
    if ckpt and ckpt.model_checkpoint_path:
        saver.restore(sess, ckpt.model_checkpoint_path)
    
    # Mot vai variable
    initial_step = model_vars['global_step'].eval()
    num_train, num_test = data.stats()
    print('num_train', num_train, 
          'num_test', num_test, 
          'n_batch',int(train_params['n_epochs'] * num_train/train_params['batch_size']), 
          'batch_per_epochs', int(num_train/train_params['batch_size']))
    train_batch = data.train_batch_generator(train_params['batch_size'])

    # Training
    for i in range(initial_step,  int(train_params['n_epochs'] * num_train/train_params['batch_size'])):
        X_batch, Y_batch = next(train_batch)
        feed = {model_vars['X']: X_batch,
            model_vars['Y']: Y_batch}
        batch_cost, _, summary = sess.run([model_vars['cost'], 
                                                      model_vars['optimizer'], 
                                                      model_vars['summary_op']], feed)
        writer.add_summary(summary, global_step=i)

        if i % train_params['skip_step'] == 0:
            print('batch', i, 'cost', batch_cost)
            saver.save(sess, './checkpoints/', i)


In [ ]:
train_params = {
    'batch_size' : 2,
    'skip_step' : 1,
    'n_epochs' : 10000000
}

with tf.Session(graph=graph) as sess:
    tf.global_variables_initializer().run()
    X_batch, seq_len_batch, Y_batch = next(train_batch)